# 1. Загрузка датасета tiny-imagenet-200

In [3]:
import os, sys
import zipfile

from urllib.request import urlretrieve

def download_tinyImg200(path,
                        url='http://cs231n.stanford.edu/tiny-imagenet-200.zip',
                        file_name='tiny-imagenet-200.zip'):
    """
    download images from url to full_path with urlretrieve
    """
    
    if not os.path.exists(path):
        # create dir if path not exist
        os.mkdir(path)
        
    full_path = os.path.join(path, file_name)
        
    urlretrieve(url, full_path)
    
    return full_path
    
def zip_unpack(full_path):
    """
    unpack zip file from full_path
    """
    zip_ref = zipfile.ZipFile(full_path, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [ ]:
path = '/home/starkov/imagenet/'
full_path = download_tinyImg200(path)

zip_unpack(full_path)

# 1.1. Resize изображений, cv2

In [25]:
import cv2
import glob

def resize_img(image_path, size):
    img = cv2.imread(image_path)
    img = cv2.resize(img,(size,size), interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(image_path,img)

def resize_all(tiny_path='./test_tiny/',
               train_path='train/*/*/*',
               val_path='val/*/*'):
    
    train = glob.glob(f'{tiny_path}{train_path}')
    val = glob.glob(f'{tiny_path}{val_path}')
    
    for img in train:
        resize_img(img, 224)
        
    for img in val:
        resize_img(img, 224)

In [26]:
resize_all()

# 2. Pytorch

In [27]:
import torch

import torchvision
import torchvision.transforms as transforms

import os

### 2.1. Создание dataloader'ов

In [121]:
def get_dataloaders_list(tiny_test_path='./test_tiny/',
                         num_workers={'train': 0,'val': 0},
                         split_tiny_path=['train', 'val'],
                         data_transforms={'train': None, 'val': None},
                         batch_size=40):
    
    full_path = {split: os.path.join(tiny_test_path, split) for split in split_tiny_path}
    
    
    datasets = {split: torchvision.datasets.ImageFolder(full_path[split],
                                                             data_transforms[split]) for split in split_tiny_path}
    sizes = {split: len(datasets[split]) for split in split_tiny_path}
    
   
    dataloaders = {split: torch.utils.data.DataLoader(datasets[split],
                                                       batch_size=batch_size,
                                                       shuffle=True,
                                                       num_workers=num_workers[split]) for split in split_tiny_path}
    
    return [dataloaders, sizes]

In [126]:
workers = {'train': 60,
           'val': 2}

transformation = {
    'train': transforms.Compose([
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ])
}

dataloaders, sizes = get_dataloaders_list(num_workers=workers,
                                          data_transforms=transformation, 
                                          batch_size=100)

In [127]:
sizes

{'train': 100000, 'val': 10000}

### 2.2. Обучение сети

In [27]:
# в терминале:
# sudo fallocate -l 10G /swapfile
# sudo chmod 600 /swapfile
# sudo mkswap /swapfile
# sudo swapon /swapfile

In [106]:
import torchvision.models as models
import torch.optim as opt
import torch.nn as nn

import sys

In [130]:
import copy

def train_model(dataloaders,
                dataset_sizes,
                model_name='resnet18',
                optimizer_name='SGD',
                criterion=nn.CrossEntropyLoss(),
                num_epochs=10,
                pretrained=False,
                **opt_params):
    
    model = models.__dict__[model_name](pretrained=pretrained)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.fc.out_features = 200
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    optimizer = opt.__dict__[optimizer_name](model.parameters(), **opt_params)

    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best = 0
    
    for epoch in range(num_epochs):
        
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            
            for i,(inputs, labels) in enumerate(dataloaders[phase]):
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                print(f'\rIter: {i+1}/{len(dataloaders[phase])} current loss: {loss.item() * inputs.size(0)}', 
                      end='', 
                      flush=True)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'train':
                avg_loss = epoch_loss
                t_acc = epoch_acc
            else:
                val_loss = epoch_loss
                val_acc = epoch_acc
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best = epoch + 1
                best_model = copy.deepcopy(model.state_dict())
        
        print(f'Train ---- loss: {avg_loss} accuracy: {t_acc*100}%')
        print(f'Validation ---- loss: {val_loss} accuracy: {val_acc*100}%')
        print()
    
    
    print(f'Validation best accuracy: {best_acc*100}%')
    
    return best_model

In [131]:
best_model = train_model(dataloaders,
                         sizes,
                         num_epochs=10, 
                         pretrained=False,
                         lr=0.001,
                         momentum=0.9)

Epoch 1/10
----------
Iter: 100/100 current loss: 467.359113693237383Train ---- loss: 5.268958933830262 accuracy: 3.314%
Validation ---- loss: 4.717855415344238 accuracy: 6.13%

Epoch 2/10
----------
Iter: 100/100 current loss: 405.026578903198248Train ---- loss: 4.51236998796463 accuracy: 8.045%
Validation ---- loss: 4.29373037815094 accuracy: 10.440000000000001%

Epoch 3/10
----------
Iter: 100/100 current loss: 409.9702358245849634Train ---- loss: 4.164669118165969 accuracy: 11.999%
Validation ---- loss: 4.088132531642914 accuracy: 12.41%

Epoch 4/10
----------
Iter: 100/100 current loss: 364.5824909210205096Train ---- loss: 3.9448413727283476 accuracy: 14.953%
Validation ---- loss: 3.835593304634094 accuracy: 16.05%

Epoch 5/10
----------
Iter: 100/100 current loss: 373.5578060150146534Train ---- loss: 3.7851715466976166 accuracy: 17.321%
Validation ---- loss: 3.7537241864204405 accuracy: 17.27%

Epoch 6/10
----------
Iter: 100/100 current loss: 366.2917852401733453Train ---- loss:

In [132]:
best_model = train_model(dataloaders,
                         sizes,
                         num_epochs=10, 
                         pretrained=True,
                         lr=0.001,
                         momentum=0.9)

Epoch 1/10
----------
Iter: 100/100 current loss: 308.981752395629987Train ---- loss: 4.3620688700675965 accuracy: 15.023%
Validation ---- loss: 3.049084939956665 accuracy: 30.29%

Epoch 2/10
----------
Iter: 100/100 current loss: 197.3064064979553264Train ---- loss: 2.5109217983484267 accuracy: 41.064%
Validation ---- loss: 2.1105877387523653 accuracy: 48.52%

Epoch 3/10
----------
Iter: 100/100 current loss: 152.3953080177307915Train ---- loss: 1.927002214550972 accuracy: 53.052%
Validation ---- loss: 1.7673091435432433 accuracy: 56.089999999999996%

Epoch 4/10
----------
Iter: 100/100 current loss: 165.3261423110962724Train ---- loss: 1.6266151379346847 accuracy: 59.631%
Validation ---- loss: 1.5819066548347473 accuracy: 60.150000000000006%

Epoch 5/10
----------
Iter: 100/100 current loss: 153.193891048431466Train ---- loss: 1.4281135547757149 accuracy: 64.059%
Validation ---- loss: 1.4637005400657654 accuracy: 62.760000000000005%

Epoch 6/10
----------
Iter: 100/100 current loss: 